# Q-Learning con FrozenLake-v1 ⛄ and Taxi-v3 🚕

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/envs.gif" alt="Environments"/>

### 🎮 Entornos:

- [FrozenLake-v1](https://gymnasium.farama.org/environments/toy_text/frozen_lake/)
- [Taxi-v3](https://gymnasium.farama.org/environments/toy_text/taxi/)

### 📚 RL-Library:

- Python and NumPy
- [Gymnasium](https://gymnasium.farama.org/)

## Un pequeño resumen de Q-Learning

*Q-Learning* **es un algoritmo de aprendizaje por refuerzo que**:

- Aprende la función-Q, una función acción-valor que codifica una tabla Q que contiene todos los valores de los pares estado-acción de un problema.


- Dado un estado y una acción, nuestra función Q buscará en la tabla Q el valor correspondiente
    
<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-function-2.jpg" alt="Q function"  width="100%"/>

- Una vez finalizado el entrenamiento, tendremos una función Q óptima y, por tanto, una tabla Q óptima.
    
- Y si tenemos una función Q óptima, tenemos una política óptima, ya que sabemos para, cada estado, la mejor acción a tomar.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/link-value-policy.jpg" alt="Link value policy"  width="100%"/>


Pero, al principio, la Q-Table no es útil ya que da un valor arbitrario para cada par estado-acción (la mayoría de las veces inicializamos la Q-Table con valores 0). Pero, a medida que exploremos el entorno y actualicemos nuestra Q-Table nos dará mejores aproximaciones

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/q-learning.jpeg" alt="q-learning.jpeg" width="100%"/>

Aquí tienes el pseudocódigo para Q-Learning:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" alt="Q-Learning" width="100%"/>

## Instalar dependencias y crear un display virtual 🔽

En el notebook, vamos a necesitar generar un vídeo para su visualización. Para ello, con Colab, **necesitamos disponer de una pantalla virtual para renderizar el entorno** (y así grabar la secuencia de las simulaciones).

Tenemos que instalar:

- `gymnasium`: Contiene los entornos FrozenLake-v1 ⛄ y Taxi-v3 🚕.
- `pygame`: Necesarios para el interfaz gráfico de FrozenLake-v1 y Taxi-v3.
- `numpy`: Necesario para gestionar la Q-table.


In [1]:
!pip install gymnasium
!pip install pygame
!pip install mediapy # Para reproducir vídeos

     |████████████████████████████████| 953 kB 3.6 MB/s eta 0:00:01
     |████████████████████████████████| 12.3 MB 4.8 MB/s eta 0:00:01


In [2]:
!sudo apt-get update
!sudo apt-get install -y python3-opengl
!apt install ffmpeg xvfb
!pip3 install pyvirtualdisplay

Password:
sudo: a password is required
Password:

Para asegurarse de que se utilizan las nuevas bibliotecas instaladas, a veces es necesario reiniciar el runtime del entorno. La siguiente celda forzará al runtime a bloquearse, por lo que tendrá que reconectarse de nuevo. Gracias a este truco, podremos ejecutar el display virtual.

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
# Virtual display
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

## Importamos los paquetes 📦

In addition to the installed libraries, we also use:

- `random`: generar números aleatorios (será útilo para la política epsilon-greedy).
- `imageio`: Manejar el vídeo que hagamos.

In [ ]:
import numpy as np
import gymnasium as gym
import random
import imageio
import os
import tqdm

from tqdm.notebook import tqdm

# Parte 1: Frozen Lake ⛄ (versión no resbaladiza)

## Entendiendo [Entorno FrozenLake ⛄]((https://gymnasium.farama.org/environments/toy_text/frozen_lake/)
---

💡 Una buena costumbre cuando empiezas a usar un entorno es consultar su documentación

👉 https://gymnasium.farama.org/environments/toy_text/frozen_lake/

---

Vamos a entrenar a nuestro agente Q-Learning **para que navegue desde el estado inicial (S) hasta el estado meta (G) caminando sólo sobre baldosas congeladas (F) y evitando los agujeros (H)**.

Podemos tener dos tamaños de entorno:

- `map_name="4x4"`: una versión de cuadrícula 4x4
- `map_name="8x8"`: una versión en cuadrícula de 8x8


El entorno tiene dos modos:

- `is_slippery=False`: El agente siempre se mueve **en la dirección prevista** debido a la naturaleza no resbaladiza del lago helado (determinista).
- `is_slippery=True`: El agente **puede no moverse siempre en la dirección prevista** debido a la naturaleza resbaladiza del lago helado (estocástico).

### Solución

In [ ]:
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=False, render_mode="rgb_array")

Podemos crear nuestro propio grid como se muestra a continuación:

```python
desc=["SFFF", "FHFH", "FFFH", "HFFG"]
gym.make('FrozenLake-v1', desc=desc, is_slippery=False)
```

por ahora, cogemos el entorno por defecto.


### Veamos cómo es el entorno:


In [ ]:
# We create our environment with gym.make("<name_of_the_environment>")- `is_slippery=False`: The agent always moves in the intended direction due to the non-slippery nature of the frozen lake (deterministic).
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space", env.observation_space)
print("Sample observation", env.observation_space.sample()) # Get a random observation

`Observation Space Shape Discrete(16)` indica que la observación es un entero que representa la **posición actual del agente como fila_actual * ncols + col_actual (donde tanto la fila como la col empiezan en 0)**.

Por ejemplo, la posición de la meta en el mapa 4x4 se puede calcular de la siguiente manera: 3 * 4 + 3 = 15. El número de observaciones posibles depende del tamaño del mapa. **Por ejemplo, el mapa 4x4 tiene 16 observaciones posibles**.

Por ejemplo, esto es lo que representaría con `state = 0`:

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/frozenlake.png" alt="FrozenLake">


In [ ]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.n)
print("Action Space Sample", env.action_space.sample()) # Take a random action

El espacio de acciones (el conjunto de acciones posibles que puede realizar el agente) es discreto con 4 acciones disponibles 🎮:
- 0: IR A LA IZQUIERDA
- 1: IR ABAJO
- 2: IR A LA DERECHA
- 3: IR ARRIBA

Función de recompensa 💰:
- Llegar a la meta: +1
- Caer en un agujero: 0
- Quedarte congelado: 0

## Crear e inicializar la tabla Q 🗄️

(👀 Paso 1 del pseudocódigo)

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" alt="Q-Aprendizaje" width="100%"/>


Para saber cuántas filas (estados) y columnas (acciones) a utilizar, necesitamos saber las acciones y el número de estados del problema. Ya conocemos sus valores de antes, pero vamos a querer obtenerlos para que el algoritmo se generalice para diferentes entornos. Gym nos proporciona una forma de hacerlo: `env.espacio_acción.n` y `env.espacio_observacion.n`.


### Solución

In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

action_space = env.action_space.n
print("There are ", action_space, " possible actions")

In [ ]:
# Let's create our Qtable of size (state_space, action_space) and initialized each values at 0 using np.zeros
def initialize_q_table(state_space, action_space):
  Qtable = np.zeros((state_space, action_space))
  return Qtable

In [ ]:
Qtable_frozenlake = initialize_q_table(state_space, action_space)

## Definir la mejor política 🤖

Recuerda que tenemos dos políticas ya que Q-Learning es un algoritmo **off-policy**. Esto significa que estamos usando una **política diferente para actuar y actualizar la función de valor**.

- Política epsilon-greedy (exploración)
- Política greedy (explotación)

La política greedy también será la política final que tendremos cuando el agente Q-learning complete el entrenamiento. La política greedy se utiliza para seleccionar una acción utilizando la tabla Q.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/off-on-4.jpg" alt="Q-Learning" width="100%"/>


#### Solución

In [ ]:
def greedy_policy(Qtable, state):
  # Exploitation: take the action with the highest state, action value
  action = np.argmax(Qtable[state][:])

  return action

## Definición de la política epsilon-greedy 🤖

In [ ]:
def epsilon_greedy_policy(Qtable, state, epsilon):
  # Randomly generate a number between 0 and 1
  random_num = random.uniform(0,1)
  # if random_num > greater than epsilon --> exploitation
  if random_num > epsilon:
    # Take the action with the highest value given a state
    # np.argmax can be useful here
    action = greedy_policy(Qtable, state)
  # else --> exploration
  else:
    action = env.action_space.sample()

  return action

## Definición de hiperparámetros ⚙️

Los hiperparámetros relacionados con la exploración son algunos de los más importantes.

- Tenemos que asegurarnos de que nuestro agente **explora lo suficiente del espacio de estados** para aprender una buena aproximación de valores. Para ello, necesitamos tener un decaimiento progresivo del épsilon.
- Si disminuimos epsilon demasiado rápido (tasa de decaimiento demasiado alta), **corremos el riesgo de que nuestro agente se quede atascado**, ya que no ha explorado lo suficiente el espacio de estados y, por tanto, no puede resolver el problema.


In [ ]:
# Training parameters
n_training_episodes = 10000  # Total training episodes
learning_rate = 0.7          # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# Environment parameters
env_id = "FrozenLake-v1"     # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate
eval_seed = []               # The evaluation seed of the environment

# Exploration parameters
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05            # Minimum exploration probability
decay_rate = 0.0005            # Exponential decay rate for exploration prob

## El bucle de entrenamiento

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/unit3/Q-learning-2.jpg" alt="Q-Learning" width="100%"/>

El bucle de entrenamiento sería como lo siguiente:

```
Por cada episodio:

Reducimos epsilon (ya que cada vez se necesita menos exploración)
Restablecer el entorno

  Para cada paso dentro del número máximo (`max_steps`):
    Eleginos la acción At utilizando la política epsilon-greedy
    Realiza la acción (a) y observa el estado resultante (s') y la recompensa (r)
    Actualizar el Q-valor Q(s,a) utilizando la ecuación de Bellman Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
    Si hemos llegado a un estado final, terminar el episodio
    El siguiente estado es el nuevo estado s'
```

#### Solución

In [ ]:
def train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable):
  for episode in tqdm(range(n_training_episodes)):
    # Reduce epsilon (because we need less and less exploration)
    epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
    # Reset the environment
    state, info = env.reset()
    step = 0
    terminated = False
    truncated = False

    # repeat
    for step in range(max_steps):
      # Choose the action At using epsilon greedy policy
      action = epsilon_greedy_policy(Qtable, state, epsilon)

      # Take action At and observe Rt+1 and St+1
      # Take the action (a) and observe the outcome state(s') and reward (r)
      new_state, reward, terminated, truncated, info = env.step(action)

      # Update Q(s,a):= Q(s,a) + lr [R(s,a) + gamma * max Q(s',a') - Q(s,a)]
      Qtable[state][action] = Qtable[state][action] + learning_rate * (reward + gamma * np.max(Qtable[new_state]) - Qtable[state][action])

      # If terminated or truncated finish the episode
      if terminated or truncated:
        break

      # Our next state is the new state
      state = new_state
  return Qtable

## Entrenando el agente Q-Learning 🏃

In [ ]:
Qtable_frozenlake = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_frozenlake)

## Veamos como queda la Q-tabla 👀

In [ ]:
Qtable_frozenlake

## El método de evaluación 📝

- Definimos el método de evaluación que vamos a utilizar para probar nuestro agente Q-Learning.

In [ ]:
def evaluate_agent(env, max_steps, n_eval_episodes, Q, seed):
  """
  Evaluate the agent for ``n_eval_episodes`` episodes and returns average reward and std of reward.
  :param env: The evaluation environment
  :param max_steps: Maximum number of steps per episode
  :param n_eval_episodes: Number of episode to evaluate the agent
  :param Q: The Q-table
  :param seed: The evaluation seed array (for taxi-v3)
  """
  episode_rewards = []
  for episode in tqdm(range(n_eval_episodes)):
    if seed:
      state, info = env.reset(seed=seed[episode])
    else:
      state, info = env.reset()
    step = 0
    truncated = False
    terminated = False
    total_rewards_ep = 0

    for step in range(max_steps):
      # Take the action (index) that have the maximum expected future reward given that state
      action = greedy_policy(Q, state)
      new_state, reward, terminated, truncated, info = env.step(action)
      total_rewards_ep += reward

      if terminated or truncated:
        break
      state = new_state
    episode_rewards.append(total_rewards_ep)
  mean_reward = np.mean(episode_rewards)
  std_reward = np.std(episode_rewards)

  return mean_reward, std_reward

## Evaluar nuestro agente Q-Learning 📈

- Normalmente, debería tener una recompensa media de 1.0
- El **entorno es relativamente fácil** ya que el espacio de estados es realmente pequeño (16). Lo que puedes intentar es [sustituirlo por la versión resbaladiza](https://gymnasium.farama.org/environments/toy_text/frozen_lake/), que introduce estocasticidad, haciendo el entorno más complejo.

In [ ]:
# Evaluate our Agent
mean_reward, std_reward = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_frozenlake, eval_seed)
print(f"Mean_reward={mean_reward:.2f} +/- {std_reward:.2f}")

#### No modificar este código

In [ ]:
def record_video(env, Qtable, out_directory, fps=1, max_iter=100):
  """
  Generate a replay video of the agent
  :param env
  :param Qtable: Qtable of our agent
  :param out_directory
  :param fps: how many frame per seconds (with taxi-v3 and frozenlake-v1 we use 1)
  """
  images = []
  terminated = False
  truncated = False
  state, info = env.reset(seed=random.randint(0,500))
  img = env.render()
  images.append(img)
  iters = 0
  while iters<max_iter and (not terminated or truncated):
    # Take the action (index) that have the maximum expected future reward given that state
    action = np.argmax(Qtable[state][:])
    state, reward, terminated, truncated, info = env.step(action) # We directly put next_state = state for recording logic
    img = env.render()
    images.append(img)
    iters = iters+1
  imageio.mimsave(out_directory, [np.array(img) for i, img in enumerate(images)], fps=fps)

- Vamos a crear **el diccionario del modelo que contiene los hiperparámetros y la tabla Q**.

In [ ]:
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,

    "learning_rate": learning_rate,
    "gamma": gamma,

    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,

    "qtable": Qtable_frozenlake
}

In [ ]:
evaluate_agent(env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"])

readme_path = "."
# Step 6: Record a video
video_path = "replay.mp4"
record_video(env, model["qtable"], video_path, 10)

In [ ]:
import mediapy as media
video = media.read_video('replay.mp4')
media.show_video(video)

# Parte 2: Taxi-v3 🚖

## Entendiendo [Taxi-v3 🚕](https://gymnasium.farama.org/environments/toy_text/taxi/)
---

💡 Una buena costumbre cuando se empieza a utilizar un entorno es consultar su documentación

👉 https://gymnasium.farama.org/environments/toy_text/taxi/

---
En `Taxi-v3` 🚕, hay cuatro lugares designados en el mundo cuadriculado indicados por R(ed), G(reen), Y(ellow) y B(lue).

Cuando comienza el episodio, **el taxi parte de una casilla aleatoria** y el pasajero se encuentra en un lugar aleatorio. El taxi se dirige al lugar donde se encuentra el pasajero, **recoge al pasajero**, se dirige al destino del pasajero (otro de los cuatro lugares especificados) y **deja al pasajero**. Una vez que se deja al pasajero, el episodio termina.

<img src="https://huggingface.co/datasets/huggingface-deep-rl-course/course-images/resolve/main/en/notebooks/unit2/taxi.png" alt="Taxi">


In [ ]:
env = gym.make("Taxi-v3", render_mode="rgb_array")

Hay **500 estados discretos, ya que hay 25 posiciones del taxi, 5 posibles ubicaciones del pasajero** (incluido el caso en que el pasajero está en el taxi) y **4 ubicaciones de destino.**

In [ ]:
state_space = env.observation_space.n
print("There are ", state_space, " possible states")

In [ ]:
action_space = env.action_space.n
print("There are ", action_space, " possible actions")

El espacio de acciones (el conjunto de acciones posibles que puede realizar el agente) es discreto con **6 acciones disponibles 🎮**:

- 0: desplazarse al sur
- 1: hacia el norte
- 2: desplazarse al este
- 3: desplazarse al oeste
- 4: recoger pasajero
- 5: dejar pasajero

Función de recompensa 💰:

- -1 por paso, a menos que se active otra recompensa.
- +20 por entregar pasajero.
- -10 ejecutando acciones "recoger" y "dejar" ilegalmente.

In [ ]:
# Create our Q table with state_size rows and action_size columns (500x6)
Qtable_taxi = initialize_q_table(state_space, action_space)
print(Qtable_taxi)
print("Q-table shape: ", Qtable_taxi .shape)

## Definir los hiperparámetros ⚙️

⚠ NO MODIFICAR EVAL_SEED: el array eval_seed **nos permite evaluar un agente con las mismas posiciones de partida de taxi para cada ejecución**

In [ ]:
# Training parameters
n_training_episodes = 25000   # Total training episodes
learning_rate = 0.7           # Learning rate

# Evaluation parameters
n_eval_episodes = 100        # Total number of test episodes

# DO NOT MODIFY EVAL_SEED
eval_seed = [16,54,165,177,191,191,120,80,149,178,48,38,6,125,174,73,50,172,100,148,146,6,25,40,68,148,49,167,9,97,164,176,61,7,54,55,
 161,131,184,51,170,12,120,113,95,126,51,98,36,135,54,82,45,95,89,59,95,124,9,113,58,85,51,134,121,169,105,21,30,11,50,65,12,43,82,145,152,97,106,55,31,85,38,
 112,102,168,123,97,21,83,158,26,80,63,5,81,32,11,28,148] # Evaluation seed, this ensures that all classmates agents are trained on the same taxi starting position
                                                          # Each seed has a specific starting state

# Environment parameters
env_id = "Taxi-v3"           # Name of the environment
max_steps = 99               # Max steps per episode
gamma = 0.95                 # Discounting rate

# Exploration parameters
max_epsilon = 1.0             # Exploration probability at start
min_epsilon = 0.05           # Minimum exploration probability
decay_rate = 0.005            # Exponential decay rate for exploration prob

## Entrenamos el agente Q-Learning

In [ ]:
Qtable_taxi = train(n_training_episodes, min_epsilon, max_epsilon, decay_rate, env, max_steps, Qtable_taxi)
Qtable_taxi

Evaluamos el agente Q-Learning

In [ ]:
# Evaluate our Agent
mean_reward_taxi, std_reward_taxi = evaluate_agent(env, max_steps, n_eval_episodes, Qtable_taxi, eval_seed)
print(f"Mean_reward={mean_reward_taxi:.2f} +/- {std_reward_taxi:.2f}")

In [ ]:
model = {
    "env_id": env_id,
    "max_steps": max_steps,
    "n_training_episodes": n_training_episodes,
    "n_eval_episodes": n_eval_episodes,
    "eval_seed": eval_seed,

    "learning_rate": learning_rate,
    "gamma": gamma,

    "max_epsilon": max_epsilon,
    "min_epsilon": min_epsilon,
    "decay_rate": decay_rate,

    "qtable": Qtable_taxi
}

In [ ]:
#evaluate_agent(env, model["max_steps"], model["n_eval_episodes"], model["qtable"], model["eval_seed"])

readme_path = "."
# Step 6: Record a video
video_path = "replay_taxi.mp4"
record_video(env, model["qtable"], video_path, 2)

In [ ]:
import mediapy as media
video = media.read_video('replay_taxi.mp4')
media.show_video(video)